In [156]:
import numpy as np
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import pandas as pd
import json
import os
import requests
import time
from tqdm import tqdm

In [157]:
url = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/APOTEK_20200109.csv'
exceldata = pd.read_csv(url)
exceldata = exceldata.dropna(subset=['Adresse', 'PostNr'])
df=exceldata.reset_index(drop=True)
df['Navn_lille']=[x.casefold() for x in df['Navn']]
df.head()

,ApotekNr,Navn,TypeID,Adresse,PostNr,Startdato,Slutdato,amt,Kommune,Region,Navn_lille
0,359,Albertslund Apotek,6,Bytorvet 5,2620.0,01Jan1966 0:00:00.000,NaN,1084.0,165.0,1084.0,albertslund apotek
1,346-1,Allerød Apotek,7,Allerød Stationsvej 9,3450.0,NaN,NaN,1084.0,201.0,1084.0,allerød apotek
2,082-1,Allinge Apotek,7,Kirkegade 13,3770.0,01May2016 0:00:00.000,NaN,1084.0,400.0,1084.0,allinge apotek
3,175,Allinge Apotek,6,Kirkegade 13,3770.0,01-01-1898 00:00,01May2016 0:00:00.000,1084.0,400.0,1084.0,allinge apotek
4,243-1,Amager Strand Apotek,7,Strandlodsvej 41,2300.0,21Sep2015 0:00:00.000,NaN,1084.0,101.0,1084.0,amager strand apotek


In [158]:
import math
# List for storing names of pharmacies missing start date 
mangler_start = []

# Looping through our dataframe to identify 'nan' in start date
for i in range(len(df)):
    # Using try so we skip all the strings in the column * note that the NaN values are float but the dates are str
    try:
        if math.isnan(df['Startdato'][i]):
            mangler_start.append(df['Navn_lille'][i])
    #skipping if value is not a flaot i.e. it is not nan        
    except Exception:
        continue

In [159]:
# Noting here that we are missing 153 start dates
len(mangler_start)

153

In [160]:
# Attempting to locate or missing start dates in all firms registered as pharmacies in https://datacvr.virk.dk/data/
# We start by making a very general search for "apotek*"
# We Import our search results from the datacvr.virk.dk here

# First we do it for CVR numbers:
url_virk = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/CVR_virk.csv'
cvr_virk = pd.read_csv(url_virk, sep=';')
# We note that we have 855 firms in our search result
print(cvr_virk.shape)

# Selecting only rows with 'Hovedbranche = 477300 r 523100' to remove every irrelevant firm
cvr_virk_ap = cvr_virk[(cvr_virk.Hovedbranche == '477300 Apoteker') | (cvr_virk.Hovedbranche =='523100 Apoteker')]
cvr_virk_a = cvr_virk_ap.reset_index(drop=True)
print(cvr_virk_a.shape)

(855, 12)
(378, 12)


In [161]:
# Same procedure as above for P-numbers:
# P-enheder
url_p = 'https://raw.githubusercontent.com/Oliverkobaek/thesis_2020/master/CVR_P-enheder.csv'
cvr_p = pd.read_csv(url_p, sep=';')
# We note that we have 1707 firms in our search result
print(cvr_p.shape)

# Selecting only rows with 'Hovedbranche = 477300 Apoteker' to remove every irrelevant firm
cvr_p_ap = cvr_p[(cvr_p.Hovedbranche == '477300 Apoteker')| (cvr_p.Hovedbranche =='523100 Apoteker')]
cvr_p_a = cvr_p_ap.reset_index(drop=True)
print(cvr_p_a.shape)

(1707, 11)
(1018, 11)


In [162]:
cvr_p_a.head(3)

,P-nummer,Startdato,Ophørsdato,Navn,Adresse,Postnr.,By,Hovedbranche,Telefonnr,Email,Reklamebeskyttet
0,1023758268,18.08.2018,NaN,Skalborg Apotek,Otto Mønsteds Vej 1,9200.0,Aalborg,477300 Apoteker,21608664.0,063fo@apoteket.dk,Ja
1,1025913309,01.09.2020,NaN,Stensballe Apotek,c/o Superbrugsen Stensballe - Sundgårdsvej 1,8700.0,Horsens,477300 Apoteker,76254200.0,NaN,Ja
2,1008657978,01.01.2000,NaN,Hasseris apotek,Hasseris Bymidte 9,9000.0,Aalborg,477300 Apoteker,NaN,NaN,Nej


In [163]:
# Caseless matching
cvr_p_a['Navn_lille']=[x.casefold() for x in cvr_p_a.Navn]
cvr_virk_a['Navn_lille']=[x.casefold() for x in cvr_virk_a.Navn]

In [164]:
print(len(cvr_p_a['Navn_lille']), len(cvr_virk_a['Navn_lille']))
samlet_opslag = list(cvr_p_a['Navn_lille'])+list(cvr_virk_a['Navn_lille'])
print(len(samlet_opslag))

1018 378
1396


In [165]:
# Matching mangler_start_lower with the data readily found in CVR
found=[]
still_missing=[]
for missing in mangler_start:
    if missing in samlet_opslag:
        found.append(missing)
    if missing not in samlet_opslag:
        still_missing.append(missing)
        
print('fundet', len(found), 'mangler endnu', len(still_missing))

fundet 87 mangler endnu 66


In [166]:
# Second matching where the name can be just a part of the entire name:
matching = {}
for i in range(len(still_missing)):
    for name in range(len(samlet_opslag)):
        if still_missing[i] in samlet_opslag[name]:
            matching[still_missing[i]]=samlet_opslag[name]

matching

{'brørup apotek': 'brørup apotek v/kurt jensen høy',
 'espergærde apotek': 'espergærde apotek, espergærde center',
 'folkebo apoteksudsalg': 'folkebo apoteksudsalg v/bente møgelhøj',
 'ishøj apotek': 'ishøj apotek v/lars østergaard christensen',
 'kolind apotek': 'kolind apoteksudsalg v/lægemiddelstyrelsen',
 'køge torvets apotek': 'køge torvets apotek v/peter schjørring-thyssen ',
 'ryomgård apoteksudsalg': 'ryomgård apoteksudsalg v/lægemiddelstyrelsen'}

In [167]:
print('fundet', len(found), 'mangler endnu', len(still_missing))
for x in matching:
    if x in still_missing:
        still_missing.remove(x)
        found.append(x)
    
print('fundet', len(found), 'mangler endnu', len(still_missing))    

fundet 87 mangler endnu 66
fundet 94 mangler endnu 59


In [168]:
# To find the final 59 we attempt scraping the CVR pages of every P-number and CVR-number we have found in this category

cvr_p_a['P-nummer']


url_p = []
for i in range(len(cvr_p_a['P-nummer'])):
    url_p.append('https://datacvr.virk.dk/data/visenhed?enhedstype=produktionsenhed&id='+str(cvr_p_a['P-nummer'][i])+'&soeg='+str(cvr_p_a['P-nummer'][i])+'&language=da')

url_v = []
for i in range(len(cvr_virk_a['CVR-nummer'])):
    url_v.append('https://datacvr.virk.dk/data/visenhed?enhedstype=virksomhed&id='+str(cvr_virk_a['CVR-nummer'][i])+'&soeg='+str(cvr_virk_a['CVR-nummer'][i])+'&language=da')


cvr_links_final = url_v + url_p

In [58]:
# We scrape these 1396 links for the historical tables that contains previous firms with start/end date and name
cvr_links = cvr_links_final

dataframe_cvr = []

for link in tqdm(cvr_links):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    table_div = soup.find_all('div', class_='table')
    for div in table_div:
        table_hist=div.find('div', {"class":'historisk-data'})
        sup_soup = BeautifulSoup(str(table_hist))
        tds = sup_soup.find_all('div', class_="row historisk-row")
        cvr_data = []
        for td in tds[1:]:
            cells = td.find_all('div', {"data-pdf-class":'column4'})
            cvr_data.append([cell.text.strip('\n')[:-1] for cell in cells])
        
    dataframe_cvr.append(pd.DataFrame(cvr_data))
    # We ended up letting it run over night with a long sleep timer as not to be blocked as bots
    time.sleep(20)
    
data_cvr = pd.concat(dataframe_cvr)

  0%|                                                                                         | 0/1396 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [242]:
# Saving the results
#data_cvr.to_excel('CVR_scraped_resultat_succes.xlsx')

In [174]:
# Loading the succesfully scraped data
data_cvr_load = pd.read_excel('CVR_scraped_resultat_succes.xlsx')

In [175]:
# implementing a lowercase version of names
data_cvr_load[3] = [x.casefold() for x in data_cvr_load[2]]

In [236]:
matching_2 = {}
for i in range(len(still_missing)):
    for name in range(len(data_cvr_load[3])):
        if still_missing[i] in data_cvr_load[3][name]:
            matching_2[still_missing[i]]=data_cvr_load[3][name]

# Den er tom lige nu fordi vi har kørt nedenstående kode:            
matching_2
# Combining matching dicts
matched = {**matching, **matching_2}

In [177]:
# Ejecting found from missing
print('fundet', len(found), 'mangler endnu', len(still_missing))
for x in matching_2:
    if x in still_missing:
        still_missing.remove(x)
        found.append(x)
    
print('fundet', len(found), 'mangler endnu', len(still_missing))  

fundet 94 mangler endnu 59
fundet 118 mangler endnu 35


In [237]:
datoer_list_p = []
datoer_list_v = []
for i in tqdm(found):

    datoer_list_p.append(cvr_p_a.loc[cvr_p_a['Navn_lille'] == i])
    datoer_list_v.append(cvr_virk_a.loc[cvr_virk_a['Navn_lille'] == i])
    
datoer_data_v = pd.concat(datoer_list_v)
datoer_data_p = pd.concat(datoer_list_p)

for key in matched:
    datoer_list_p.append(cvr_p_a.loc[cvr_p_a['Navn_lille'] == matched[key]])
    datoer_list_v.append(cvr_virk_a.loc[cvr_virk_a['Navn_lille'] == matched[key]])
    
datoer_data_v = pd.concat(datoer_list_v)
datoer_data_p = pd.concat(datoer_list_p)

datoer_data_v.to_excel('Data_v.xlsx')
datoer_data_p.to_excel('Data_p.xlsx')


100%|███████████████████████████████████████████████████████████████████████████████| 118/118 [00:00<00:00, 821.26it/s]


In [239]:
still_missing

['apoteksudsalg fra trekroner apopotek',
 'apoteksudsalget på kingosvej',
 'bankagervejs apoteksudsalg',
 'brede apoteksudsalg',
 'erritsø apoteksudsalg',
 'fensmark apoteksudsalg',
 'frederiksberg dalgas boulevard apotek',
 'glarbjerg apoteksudsalg',
 'greve apotek (gl)',
 'hedegårdens apoteksudsalg',
 'hjortespring-centrets apoteksudsalg',
 'høje gladsaxe torv apoteksudsalg',
 'høje tåstrup apoteksudsalg',
 'hørve apoteksudsalg',
 'kirkebjerg torv apoteksudsalg',
 'kolding storcenter apoteksudsalg',
 'københavn apoteket ryparken',
 'køgevej apoteksudsalg',
 'munkebo apoteksudsalg',
 'maaløv apotek',
 'næstved sct. jørgens apotek',
 'næstved storcenter apoteksudsalg',
 'odense rosengården apotek',
 'odense vollsmose apotek',
 'prøvestenscentrets apoteksudsalg',
 'rødovrevej apoteksudsalg',
 'stenløse apotek filial',
 'storcenter nord apoteksudsalg',
 'storcenterets apoteksudsalg',
 'tilst apoteksudsalg',
 'tune apoteksudsalg',
 'valby langgade apoteksudsalg',
 'valby langgade apoteksu

In [ ]:
#Resten er håndteret manuelt for at undgå at gå in i string-matchinng og levensteihn distance

In [ ]:
#Endeligt datasæt:

In [240]:
data_start = pd.read_excel('Startdatoer_cvr.xlsx')